In [1]:
from keras import applications
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.utils.np_utils import to_categorical 
from keras.callbacks import ModelCheckpoint
from keras import Model
from keras import initializers
from keras.callbacks import LearningRateScheduler
from keras.utils import layer_utils, np_utils
from keras.applications.inception_resnet_v2 import preprocess_input
#from keras.applications.densenet import preprocess_input
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils import class_weight
import seaborn as sn
from scipy import interp
from itertools import cycle
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from scipy.sparse import csr_matrix

Using TensorFlow backend.
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
image_width = 224
image_height = 224

# Change the batchsize according to your system RAM
batch_size = 64

train_dir = "D:/retinal_data_set_visioncare/VISION_CARE/"

inceptionResNetV2_base = applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))

In [3]:
x = inceptionResNetV2_base.get_layer(index=-1).output
feature_extraction_layer = GlobalAveragePooling2D()(x)
model = Model(inputs=inceptionResNetV2_base.input, outputs=feature_extraction_layer)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_width, image_height),
        batch_size=batch_size,
        class_mode=None,  # this means our generator will only yield batches of data, no labels
        shuffle=False)

nb_train_samples = len(train_generator.filenames)  
num_classes = len(train_generator.class_indices)

# get the class lebels for the training data, in the original order  
train_labels = train_generator.classes  
   
# convert the training labels to categorical vectors  
train_labels = to_categorical(train_labels, num_classes=num_classes)

Found 183 images belonging to 5 classes.


In [5]:
# # the predict_generator method returns the output of a model, given
# # a generator that yields batches of numpy data
# bottleneck_features_train = model.predict_generator(train_generator, nb_train_samples // batch_size + 1)
# # save the output as a Numpy array
# np.save(open('C:/Users/hp/Desktop/Diabetic_retinopathy_dataset_kaggle/models/inceptionResNetV2/bottle_neck_features/bottleneck_features_train_visioncare.npy', 'wb'), bottleneck_features_train)

In [5]:
# model_check_point_loc = 'C:/Users/hp/Desktop/Diabetic_retinopathy_dataset_kaggle/models/inceptionResNetV2/densenet_deep_feature_visioncare_without_SVD_dr.h5'
# #model_checkpoint = ModelCheckpoint(model_check_point_loc, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

In [5]:
train_data = np.load(open('C:/Users/hp/Desktop/Diabetic_retinopathy_dataset_kaggle/models/inceptionResNetV2/bottle_neck_features/bottleneck_features_train_visioncare.npy', 'rb'))
#test_data = np.load(open('D:/retinal_data_set_visioncare/models/inceptionResNetV2/bottle_neck_features/bottleneck_features_test_without_aug.npy', 'rb'))

In [6]:
train_data_labels = train_generator.classes

In [7]:
# Feature Scaling - fature normalizing
scaler = StandardScaler()
X_train = scaler.fit_transform(train_data)

In [8]:
len(X_train)

183

In [9]:
train_data_labels = train_generator.classes  

X = X_train
Y = train_data_labels

In [10]:
len(Y)

183

In [11]:
np.unique(train_data_labels)

array([0, 1, 2, 3, 4])

In [12]:
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(train_data_labels)
                                               ,train_data_labels)

In [13]:
dict(enumerate(class_weight))

{0: 0.9384615384615385,
 1: 1.6636363636363636,
 2: 0.4945945945945946,
 3: 1.83,
 4: 1.3071428571428572}

In [14]:
class_weight_dic = dict(enumerate(class_weight))

In [15]:
# define 5-fold cross validation test harness
#kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
kfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=25)
cvscores = []
trainScores = []
f1Score = []
num_k_folds = 5
fold_counter = 0
val_conmats = []
val_precisions = []
val_recalls = []
val_f_scores = []
input_dim = X_train.shape[1:][0]

svm_classifier = SVC(kernel='linear', C=0.5, class_weight=class_weight_dic)

In [16]:
for train, test in kfold.split(X, Y):
    # Fit the model
    svm_classifier.fit(X[train], Y[train])
    
    y_train_pred = svm_classifier.predict(X[train])
    #y_train_pred = np.argmax(y_train_pred, axis=1)
    
    y_validation_pred = svm_classifier.predict(X[test])
    #y_validation_pred = np.argmax(y_validation_pred, axis=1)
    #y_validation_pred = np.argmax(y_validation_pred, axis=1)
    
    [precision, recall, f_score, _] = precision_recall_fscore_support(Y[test], y_validation_pred)
    #print("Validation k-fold #%d - precision: %f, recallL: %f, f-score: %f" % (fold_counter, precision, recall, f_score))
    
    conmat = confusion_matrix(Y[test], y_validation_pred)
    
    val_precisions.append(precision)
    val_recalls.append(recall)
    val_f_scores.append(f_score)
    val_conmats.append(conmat)
    fold_counter = fold_counter + 1
    
    trainScores.append(accuracy_score(Y[train], y_train_pred))
    cvscores.append(accuracy_score(Y[test], y_validation_pred))
    
print("\nAveraging the 5-fold results:")
print("%s: %.2f%%" % ('AVG Train Acc ', np.mean(trainScores) * 100))
print("%s: %.2f%%" % ('AVG Validation Acc ', np.mean(cvscores) * 100))
print("Validation precision - mean: %f, stddev: %f" % (np.mean(val_precisions), np.std(val_precisions)))
print("Validation recall - mean: %f, stddev: %f" % (np.mean(val_recalls), np.std(val_recalls)))
print("Validation f-score - mean: %f, stddev: %f" % (np.mean(val_f_scores), np.std(val_f_scores)))
print("Confusion matrix:")
print (sum(val_conmats).astype(float) / fold_counter)

c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Averaging the 5-fold results:
AVG Train Acc : 100.00%
AVG Validation Acc : 64.58%
Validation precision - mean: 0.623705, stddev: 0.240857
Validation recall - mean: 0.582657, stddev: 0.274122
Validation f-score - mean: 0.577223, stddev: 0.230484
Confusion matrix:
[[ 6.08  0.56  1.08  0.    0.08]
 [ 1.28  1.2   1.92  0.    0.  ]
 [ 0.96  1.52 11.12  0.36  0.84]
 [ 0.    0.    0.88  2.52  0.6 ]
 [ 0.36  0.12  1.4   1.    2.72]]
